In [1]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # Fetch a table# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)
sql_query = "SELECT * FROM osr"

# Execute the SQL query
cursor = mydb.cursor()
cursor.execute(sql_query)

rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=column_names)
cursor.close()
mydb.close()

In [ ]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #Only Found out OSM nodes for first 10 Trip_ids# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
df = df[df['trip_id'].between(1, 10)]
print(df)

In [ ]:
import overpass
import overpy
import json
import re
import pandas as pd
import numpy as np

dfs = []

# Initialize Overpass API
op = overpy.Overpass()

# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    trip_id = row['trip_id']
    node_ids = row['node_id'].split(',')  # Split the comma-separated node IDs 
    
    # Create a list to store dictionaries for each node in the current trip
    trip_data = []
    
    # Iterate over each node ID in the current row
    for node_id in node_ids:
        # Query Overpass API to retrieve information for the current node ID
        node_query = op.query("""
            [out:json];
            node({node_id});
            out center;
        """.format(node_id=node_id))

        # Extract information from the query response
        for node_data in node_query.nodes:
            node_info = {
                'TripID': trip_id,
                'NodeID': node_id,
                'Latitude': node_data.lat,
                'Longitude': node_data.lon,
                'Tags': node_data.tags,
                'WayID': None,
                'Way Tags': None
            }

            # Query ways associated with the current node
            ways_query = op.query("""
                [out:json];
                node({node_id});
                <;
                out;
            """.format(node_id=node_id))

            ways_data = ways_query.ways
            if ways_data:
                # Store information for each way associated with the node
                way_info_list = []
                for way in ways_data:
                    way_info_list.append({
                        'WayID': way.id,
                        'Way Tags': way.tags
                    })
                node_info['WayID'] = way_info_list[0]['WayID']  # Set the first way ID
                node_info['Way Tags'] = way_info_list[0]['Way Tags']  # Set the first way tags

            # Append the node information to the trip_data list
            trip_data.append(node_info)
    
    # Create a DataFrame for the current trip and append it to the dfs list
    trip_df = pd.DataFrame(trip_data)
    dfs.append(trip_df)

# Concatenate all DataFrames in the dfs list along rows
new_df = pd.concat(dfs, ignore_index=True)

# Print the new DataFrame
print(new_df)




# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # to find ways and nodes around a coordinate# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# # # # # Loop through rows in the original DataFrame
# for index, row in df_filtered.iterrows():
#     # Extract trip_id, latitude, and longitude from the current row
#     trip_id = row['trip_id']
#     latitude = row['latitude']
#     longitude = row['longitude']

#     roads_query = op.query("""
#         [out:json];
#         way(around:20, {lat}, {lon});
#         out center;
#         >;
#         out body;
#     """.format(lat=latitude, lon=longitude))
    
#     way_nodes = roads_query.ways
    
#     for way in roads_query.ways:

#         print(f"Way ID: {way.id}")
#         way_name = way.tags.get('name', 'N/A')
#         # print(f"Center Coordinates: {way.center_lat}, {way.center_lon}")
#         print("Nodes:")
#         for node in way.nodes:
#             node_coords = pd.DataFrame({
#                 'trip_id' : trip_id,
#                 'Node_id': node.id,
#                 'latitude': node.lat,
#                 'longitude': node.lon,
#                 'way_name': way_name,
#                 'way_nodes': way_nodes,

#             })
#             print(f"  Node ID: {node}")
#         print("\n")
        
#         way_name = way.tags.get('name', 'N/A')

In [8]:
import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)
create_table_query = """
CREATE TABLE IF NOT EXISTS osm (
    trip_id INT,
    node_id BIGINT,
    latitude FLOAT,
    longitude FLOAT,
    tags TEXT,
    way_id BIGINT,
    way_tags TEXT
)
"""

cursor = mydb.cursor()

# Execute the CREATE TABLE query
cursor.execute(create_table_query)

# Define the INSERT INTO query
insert_query = "INSERT INTO osm (trip_id, node_id, latitude, longitude, tags, way_id, way_tags) VALUES (%s, %s, %s, %s, %s, %s, %s)"

# Iterate over each row in the DataFrame and insert the values into the SQL table
for index, row in new_df.iterrows():
    # Convert tags and way_tags to JSON strings
    tags_json = json.dumps(row['Tags'])
    way_tags_json = json.dumps(row['Way_tags'])
    
    # Insert the row into the SQL table
    cursor.execute(insert_query, (row['TripID'], row['NodeID'], row['Latitude'], row['Longitude'], tags_json, row['WayID'], way_tags_json))

# Commit the changes
mydb.commit()

# Close the cursor and the connection
cursor.close()
mydb.close()


In [11]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # Viewing Table # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

import mysql.connector
import pandas as pd

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)
sql_query = "SELECT * FROM osr"

# Execute the SQL query
cursor = mydb.cursor()
cursor.execute(sql_query)

rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]

df = pd.DataFrame(rows, columns=column_names)

# batch_data = df[df['trip_id'] == 1]
# print(len(batch_data['latitude']))

print(df)
cursor.close()
mydb.close()

       trip_id                                           response  \
0            1  {"code": "Ok", "matchings": [{"confidence": 0....   
1            1  {"code": "Ok", "matchings": [{"confidence": 0....   
2            1  {"code": "Ok", "matchings": [{"confidence": 0....   
3            2  {"code": "Ok", "matchings": [{"confidence": 0....   
4            2  {"code": "Ok", "matchings": [{"confidence": 0....   
...        ...                                                ...   
17308     1603  {"code": "Ok", "matchings": [{"confidence": 0....   
17309     1603  {"code": "Ok", "matchings": [{"confidence": 0....   
17310     1603  {"code": "Ok", "matchings": [{"confidence": 0....   
17311     1604  {"code": "Ok", "matchings": [{"confidence": 0....   
17312     1604  {"code": "Ok", "matchings": [{"confidence": 0....   

                                                   speed  \
0      1.3200000524521,2.2999999523163,3.319999933242...   
1      4.1199998855591,3.8599998950958,4.25,3.17000

In [10]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # Alter Table # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 


import mysql.connector

# Connect to your MySQL database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="st"
)

# Create a cursor object
cursor = mydb.cursor()

# Execute the ALTER TABLE statement to rename the column
order_query = "SELECT * FROM osr ORDER BY trip_id ASC"

# alter_query = "ALTER TABLE userinformation CHANGE agetype cycling_freqtype TEXT"

cursor.execute(order_query)

# # Commit the changes
mydb.commit()

#Close the cursor and the connection
cursor.close()
mydb.close()

In [6]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # Creating Table # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

import mysql.connector


mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)

create_table_query = """
CREATE TABLE IF NOT EXISTS sc (
    trip_id INT,
    speed TEXT,
    recorded TEXT,
    latitude FLOAT,
    longitude FLOAT,
    node_id TEXT
)
"""

# # Execute the SQL query to create the new table
cursor = mydb.cursor()
cursor.execute(create_table_query)

# Insert data from matched_coordinates into the new "snapped" table
insert_query = "INSERT INTO sc (trip_id, speed, recorded, latitude, longitude, node_id) VALUES (%s, %s, %s, %s, %s, %s)"
for entry in matched_coordinates:
    cursor.execute(insert_query, (entry['trip_id'], entry['speed'], entry['recorded'], entry['latitude'], entry['longitude'], entry['node_id']))

# Commit changes and close the cursor and MySQL connection
mydb.commit()
cursor.close()
mydb.close()


In [2]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # Deleting Table # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)
cursor = mydb.cursor()
table_name = "sc" 

# Define the DROP TABLE query
drop_table_query = f"DROP TABLE IF EXISTS {table_name}"
cursor.execute(drop_table_query)
# Execute the DROP TABLE query
cursor.execute(drop_table_query)

# Commit the transaction
mydb.commit()

# Close cursor and MySQL connection
cursor.close()
mydb.close()

In [13]:
import mysql.connector

mydb = mysql.connector.connect(
    host= "localhost",
    user= "root",
    password= "root",
    database= "st"
)
cursor = mydb.cursor()

# Define the SQL query to retrieve table names
query = "SHOW TABLES"

# Execute the query
cursor.execute(query)

# Fetch all rows from the result set
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])


age
coord
cycling_freq
email
ethnicity
gender
income
note
note_type
noteinformation
osm
osr
rider_history
rider_type
sc
snappedcords
trip
user
userinformation


<!-- OSM API -->